# **1. Main**

In [ ]:
target_gini = 0.0
#target_gini ranges from 0 till 0.8 in steps of 0.1
dataset_name = 'MNIST'

epoch = 10




# **1B. Settings**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import scipy


from scipy.sparse import lil_matrix
from scipy.sparse import coo_matrix
from scipy.sparse import dok_matrix
#the "sparseoperations" Cython library was tested in Ubuntu 16.04. Please note that you may encounter some "solvable" issues if you compile it in Windows.
#import sparseoperations
import datetime

from scipy import sparse
import time
from sklearn import preprocessing
from scipy.io import loadmat
from scipy.io import savemat

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

from PIL import Image
from sklearn.model_selection import train_test_split
import urllib.request as urllib2
import errno
import os
import sys; sys.path.append(os.getcwd())
import argparse

from scipy.optimize import linear_sum_assignment #The linear_assignment function is deprecated in 0.21 and will be removed from 0.23, but sklearn.utils.linear_assignment_ can be replaced by scipy.optimize.linear_sum_assignment


In [ ]:
if dataset_name == 'coil20':
    # Set the path to the directory containing the PNG images
    png_dir = "/content/gdrive/MyDrive/datasets/Coil-20"
    # Initialize empty lists for images and labels
    images = []
    labels = []

    # Iterate over the PNG files in the directory
    for file_name in os.listdir(png_dir):
        if file_name.endswith(".png"):
            # Load the image
            image = Image.open(os.path.join(png_dir, file_name))
            # Convert the image to a numerical array
            image_array = np.array(image)
            # Append the image array to the images list
            images.append(image_array)
            # Extract the class label from the file name
            label = file_name.split("__")[0]  # Assuming the label is separated by double underscores (__) from the rest of the file name
            # Append the label to the labels list
            labels.append(label)

    # Create a dictionary to store the data
    data = {"images": images, "labels": labels}

    # Save the data as a .mat file
    savemat("/content/gdrive/MyDrive/datasets/Coil20.mat", data)


# **2A. Gini**

This code is written by Jesse Wijlhuizen

In [ ]:
def calculate_gini(x):
#given a certain distribution, this function calculates the gini coefficient
   total = 0
   for i, xi in enumerate(x[:-1], 1):
      total += np.sum(np.abs(xi - x[i:]))
      return total / (len(x) ** 2 * np.mean(x))

def gini_algorithm(target_gini, num_categories):
    # target_gini is the gini to be reached with the eventual distribution, num_categories is the number of categories in the eventual distribution
    print("Start gini-process")
    # Set the initial lower and upper bounds for the first element
    lower_bound = 0
    upper_bound = 100

    # Use binary search to find the appropriate value for the first element
    while True:
        mid = (lower_bound + upper_bound) / 2
        x = np.array([mid] + [1] * (num_categories - 1))
        gini_coeff = calculate_gini(x)

        if abs(gini_coeff - target_gini) < 1e-3:
            distribution = x / np.sum(x)
            print("Relative distribution over num_categories:", distribution)
            gini_coeff = calculate_gini(distribution)
            print("Gini coefficient:", gini_coeff)
            return gini_coeff, distribution

        elif gini_coeff > target_gini:
            upper_bound = mid
        else:
            lower_bound = mid


# **Other Classes.py**

This code is an adjustment of Atashgahi, Z., Sokar, G., van der Lee, T., Mocanu, E., Mocanu, D. C., Veldhuis, R., & Pechenizkiy, M. (2020). Quick and robust feature selection: the strength of energy-efficient sparse training for autoencoders. arXiv preprint arXiv:2012.00560.

In [ ]:
class Sigmoid:
    @staticmethod
    def activation(z):
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def prime(z):
        return Sigmoid.activation(z) * (1 - Sigmoid.activation(z))

class tanh:
    @staticmethod
    def activation(z):
        return (np.tanh(z))

    @staticmethod
    def prime(z):
        return 1 - tanh.activation(z) ** 2

class Relu:
    @staticmethod
    def activation(z):
        z[z < 0] = 0
        return z

    @staticmethod
    def prime(z):
        z[z < 0] = 0
        z[z > 0] = 1
        return z

class LeakyRelu:
    @staticmethod
    def activation(z):
        z[z < 0] *= 0.01
        return z

    @staticmethod
    def prime(z):
        z[z < 0] = 0.01
        z[z > 0] = 1
        return z


class Linear:
    @staticmethod
    def activation(z):
        return z

    @staticmethod
    def prime(z):
        return 1



class MSE:
    def __init__(self, activation_fn=None):
        """
        :param activation_fn: Class object of the activation function.
        """
        if activation_fn:
            self.activation_fn = activation_fn
        else:
            self.activation_fn = NoActivation

    def activation(self, z):
        return self.activation_fn.activation(z)

    @staticmethod
    def loss(y_true, y_pred):
        """
        :param y_true: (array) One hot encoded truth vector.
        :param y_pred: (array) Prediction vector
        :return: (flt)
        """
        return np.mean((y_pred - y_true) ** 2)

    @staticmethod
    def prime(y_true, y_pred):
        return y_pred - y_true

    def delta(self, y_true, y_pred):
        """
        Back propagation error delta
        :return: (array)
        """
        return self.prime(y_true, y_pred) * self.activation_fn.prime(y_pred)


class NoActivation:
    """
    This is a plugin function for no activation.
    f(x) = x * 1
    """

    @staticmethod
    def activation(z):
        """
        :param z: (array) w(x) + b
        :return: z (array)
        """
        return z

    @staticmethod
    def prime(z):
        """
        The prime of z * 1 = 1
        :param z: (array)
        :return: z': (array)
        """
        return np.ones_like(z)


# **Sparse DAE**

This code is an adjustment of Atashgahi, Z., Sokar, G., van der Lee, T., Mocanu, E., Mocanu, D. C., Veldhuis, R., & Pechenizkiy, M. (2020). Quick and robust feature selection: the strength of energy-efficient sparse training for autoencoders. arXiv preprint arXiv:2012.00560.

In [ ]:

def backpropagation_updates_Numpy(a, delta, rows, cols, out):
    for i in range(out.shape[0]):
        s = 0
        for j in range(a.shape[0]):
            s += a[j, rows[i]] * delta[j, cols[i]]
        out[i] = s / a.shape[0]

def find_first_pos(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

def find_last_pos(array, value):
    idx = (np.abs(array - value))[::-1].argmin()
    return array.shape[0] - idx

def createSparseWeights(epsilon, noRows, noCols):
    # generate an Erdos Renyi sparse weights mask
    weights = lil_matrix((noRows, noCols))
    for i in range(epsilon * (noRows + noCols)):
        weights[np.random.randint(0, noRows), np.random.randint(0, noCols)] = np.float64(np.random.randn() / 10)
    print("Create sparse matrix with ", weights.getnnz(), " connections and ",
          (weights.getnnz() / (noRows * noCols)) * 100, "% density level")
    weights = weights.tocsr()
    return weights

def array_intersect(A, B):
    # this are for array intersection
    nrows, ncols = A.shape
    dtype = {'names': ['f{}'.format(i) for i in range(ncols)], 'formats': ncols * [A.dtype]}
    return np.in1d(A.view(dtype), B.view(dtype))  # boolean return

class Sparse_DAE:
    def __init__(self, dimensions, activations, epsilon=20):
        self.n_layers = len(dimensions)
        self.loss = None
        self.learning_rate = None
        self.momentum = None
        self.weight_decay = None
        self.epsilon = epsilon  # control the sparsity level as discussed in the paper
        self.zeta = None  # the fraction of the weights removed
        self.droprate = 0  # dropout rate
        self.dimensions = dimensions

        # Weights and biases are initiated by index. For a one hidden layer net you will have a w[1] and w[2]
        self.w = {}
        self.b = {}
        self.pdw = {}
        self.pdd = {}

        # Activations are also initiated by index. For the example we will have activations[2] and activations[3]
        self.activations = {}
        for i in range(len(dimensions) - 1):
            if (i<len(dimensions) - 2):
                self.w[i + 1] = createSparseWeights(self.epsilon, dimensions[i],
                                                dimensions[i + 1])  # create sparse weight matrices
            else:
                self.w[i + 1] = createSparseWeights(self.epsilon, dimensions[i],
                                                dimensions[i + 1])  # create sparse weight matrices

            self.b[i + 1] = np.zeros(dimensions[i + 1])
            self.activations[i + 2] = activations[i]


    def _feed_forward(self, x, drop=False):
        # w(x) + b
        z = {}
        # activations: f(z)
        a = {1: x}  # First layer has no activations as input. The input x is the input.

        for i in range(1, self.n_layers):
            z[i + 1] = a[i] @ self.w[i] + self.b[i]
            if (drop == False):
                if (i > 1):
                    z[i + 1] = z[i + 1] * (1 - self.droprate)
            a[i + 1] = self.activations[i + 1].activation(z[i + 1])
            if (drop):
                if (i < self.n_layers - 1):
                    dropMask = np.random.rand(a[i + 1].shape[0], a[i + 1].shape[1])
                    dropMask[dropMask >= self.droprate] = 1
                    dropMask[dropMask < self.droprate] = 0
                    a[i + 1] = dropMask * a[i + 1]

        return z, a

    def _back_prop(self, z, a, adjusted_X_train):
        delta = self.loss.delta(adjusted_X_train, a[self.n_layers])
        dw = coo_matrix(self.w[self.n_layers - 1])

        # compute backpropagation updates
        #sparseoperations.backpropagation_updates_Cython(a[self.n_layers - 1],delta,dw.row,dw.col,dw.data)# If you have problems with Cython please use the backpropagation_updates_Numpy method by uncommenting the line below and commenting the one above. Please note that the running time will be much higher
        # If you have problems with Cython please use the backpropagation_updates_Numpy method by uncommenting the line below and commenting the one above. Please note that the running time will be much higher
        backpropagation_updates_Numpy(a[self.n_layers - 1], delta, dw.row, dw.col, dw.data)

        update_params = {
            self.n_layers - 1: (dw.tocsr(), delta)
        }
        for i in reversed(range(2, self.n_layers)):
            delta = (delta @ self.w[i].transpose()) * self.activations[i].prime(z[i])
            dw = coo_matrix(self.w[i - 1])

            # compute backpropagation updates
            #sparseoperations.backpropagation_updates_Cython(a[i - 1], delta, dw.row, dw.col, dw.data)# If you have problems with Cython please use the backpropagation_updates_Numpy method by uncommenting the line below and commenting the one above. Please note that the running time will be much higher
            # If you have problems with Cython please use the backpropagation_updates_Numpy method by uncommenting the line below and commenting the one above. Please note that the running time will be much higher
            backpropagation_updates_Numpy(a[i - 1], delta, dw.row, dw.col, dw.data)

            update_params[i - 1] = (dw.tocsr(), delta)
            for k, v in update_params.items():
                self._update_w_b(k, v[0], v[1])

    def _update_w_b(self, index, dw, delta):
        """
        Update weights and biases.
        :param index: (int) Number of the layer
        :param dw: (array) Partial derivatives
        :param delta: (array) Delta error.
        """
        # perform the update with momentum
        if (index not in self.pdw):
            self.pdw[index] = -self.learning_rate * dw
            self.pdd[index] = - self.learning_rate * np.mean(delta, 0)
        else:
            self.pdw[index] = self.momentum * self.pdw[index] - self.learning_rate * dw
            self.pdd[index] = self.momentum * self.pdd[index] - self.learning_rate * np.mean(delta, 0)

        self.w[index] += self.pdw[index] - self.weight_decay * self.w[index]
        self.b[index] += self.pdd[index] - self.weight_decay * self.b[index]

    def fit(self, x, y_true, x_test, y_test, loss, epochs, batch_size, learning_rate=1e-3, momentum=0.9,
            weight_decay=0.00001, zeta=0.2, dropoutrate=0, testing=True, save_filename="", noise_factor = 0.2):

        if not x.shape[0] == y_true.shape[0]:
            raise ValueError("Length of x and y arrays don't match")

        # Initiate the loss object with the final activation function
        self.loss = loss(self.activations[self.n_layers])
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.zeta = zeta
        self.droprate = dropoutrate
        self.save_filename=save_filename
        self.inputLayerConnections = []
        self.inputLayerConnections.append(self.getCoreInputConnections())
        np.savez_compressed(self.save_filename + "_input_connections.npz",
                        inputLayerConnections=self.inputLayerConnections)



        metrics = np.zeros((epochs, 2))
        mean = 0; stddev = 1
        noise = noise_factor * np.random.normal(mean, stddev, (x.shape[0], x.shape[1]))
        x_noisy = x + noise
        seed = np.arange(x.shape[0])

        for i in range(epochs):
            print("----------------------------------------------------")
            print("epoch ", i)


            # Shuffle the data
            np.random.shuffle(seed)
            x_noisy_ = x_noisy[seed]
            x_ = x[seed]
            y_ = y_true[seed]
            # training
            t1 = datetime.datetime.now()
            for j in range(x.shape[0] // batch_size):
                k = j * batch_size
                l = (j + 1) * batch_size
                z, a = self._feed_forward(x_noisy_[k:l], True)
                self._back_prop(z, a, x_[k:l])
            t2 = datetime.datetime.now()
            print("Training time: ", t2 - t1)

            if (testing):
                activations_hidden_test, activations_output_test  = self.predict(x_test)
                activations_hidden_train, activations_output_train = self.predict(x)
                loss_train = self.loss.loss(x, activations_output_train)
                metrics[i, 0] = loss_train
                loss_test = self.loss.loss(x_test, activations_output_test)
                metrics[i, 1] = loss_test
                print("Loss train: ", loss_train, "; Loss test: ", loss_test)
                plt_loss(metrics[:i+1, 0], metrics[:i+1, 1], self.save_filename)

            if (i < epochs - 1):
                # do not change connectivity pattern after the last epoch
                # self.weightsEvolution_I() #this implementation is more didactic, but slow.
                w1 , w2= self.weightsEvolution_II()  # this implementation has the same behaviour as the one above, but it is much faster.



            if (self.save_filename != ""):
                np.savetxt(self.save_filename+"/metrics.txt", metrics)
        scipy.sparse.save_npz(self.save_filename+"w1", w1)
        scipy.sparse.save_npz(self.save_filename+"w2", w2)
        return metrics
    def getCoreInputConnections(self):
        values = np.sort(self.w[1].data)
        firstZeroPos = find_first_pos(values, 0)
        lastZeroPos = find_last_pos(values, 0)

        largestNegative = values[int((1 - self.zeta) * firstZeroPos)]
        smallestPositive = values[
            int(min(values.shape[0] - 1, lastZeroPos + self.zeta * (values.shape[0] - lastZeroPos)))]

        wlil = self.w[1].tolil()
        wdok = dok_matrix((self.dimensions[0], self.dimensions[1]), dtype="float64")

        # remove the weights closest to zero
        keepConnections = 0
        for ik, (row, data) in enumerate(zip(wlil.rows, wlil.data)):
            for jk, val in zip(row, data):
                if ((val < largestNegative) or (val > smallestPositive)):
                    wdok[ik, jk] = val
                    keepConnections += 1
        return wdok.tocsr().getnnz(axis=1)

    def weightsEvolution_I(self):
        # this represents the core of the SET procedure. It removes the weights closest to zero in each layer and add new random weights
        for i in range(1, self.n_layers):

            values = np.sort(self.w[i].data)
            firstZeroPos = find_first_pos(values, 0)
            lastZeroPos = find_last_pos(values, 0)

            largestNegative = values[int((1 - self.zeta) * firstZeroPos)]
            smallestPositive = values[
                int(min(values.shape[0] - 1, lastZeroPos + self.zeta * (values.shape[0] - lastZeroPos)))]

            wlil = self.w[i].tolil()
            pdwlil = self.pdw[i].tolil()
            wdok = dok_matrix((self.dimensions[i - 1], self.dimensions[i]), dtype="float64")
            pdwdok = dok_matrix((self.dimensions[i - 1], self.dimensions[i]), dtype="float64")

            # remove the weights closest to zero
            keepConnections = 0
            for ik, (row, data) in enumerate(zip(wlil.rows, wlil.data)):
                for jk, val in zip(row, data):
                    if ((val < largestNegative) or (val > smallestPositive)):
                        wdok[ik, jk] = val
                        pdwdok[ik, jk] = pdwlil[ik, jk]
                        keepConnections += 1

            # add new random connections
            for kk in range(self.w[i].data.shape[0] - keepConnections):
                ik = np.random.randint(0, self.dimensions[i - 1])
                jk = np.random.randint(0, self.dimensions[i])
                while (wdok[ik, jk] != 0):
                    ik = np.random.randint(0, self.dimensions[i - 1])
                    jk = np.random.randint(0, self.dimensions[i])
                wdok[ik, jk] = np.random.randn() / 10
                pdwdok[ik, jk] = 0

            self.pdw[i] = pdwdok.tocsr()
            self.w[i] = wdok.tocsr()

    def weightsEvolution_II(self):
        # this represents the core of the SET procedure. It removes the weights closest to zero in each layer and add new random weights
        #evolve all layers, except the one from the last hidden layer to the output layer
        for i in range(1, self.n_layers):
            # uncomment line below to stop evolution of dense weights more than 80% non-zeros
            #if(self.w[i].count_nonzero()/(self.w[i].get_shape()[0]*self.w[i].get_shape()[1]) < 0.8):
                t_ev_1 = datetime.datetime.now()
                # converting to COO form
                wcoo = self.w[i].tocoo()
                valsW = wcoo.data
                rowsW = wcoo.row
                colsW = wcoo.col

                pdcoo = self.pdw[i].tocoo()
                valsPD = pdcoo.data
                rowsPD = pdcoo.row
                colsPD = pdcoo.col
                # print("Number of non zeros in W and PD matrix before evolution in layer",i,[np.size(valsW), np.size(valsPD)])
                values = np.sort(self.w[i].data)
                firstZeroPos = find_first_pos(values, 0)
                lastZeroPos = find_last_pos(values, 0)

                largestNegative = values[int((1 - self.zeta) * firstZeroPos)]
                smallestPositive = values[
                    int(min(values.shape[0] - 1, lastZeroPos + self.zeta * (values.shape[0] - lastZeroPos)))]

                # remove the weights (W) closest to zero and modify PD as well
                valsWNew = valsW[(valsW > smallestPositive) | (valsW < largestNegative)]
                rowsWNew = rowsW[(valsW > smallestPositive) | (valsW < largestNegative)]
                colsWNew = colsW[(valsW > smallestPositive) | (valsW < largestNegative)]

                newWRowColIndex = np.stack((rowsWNew, colsWNew), axis=-1)
                oldPDRowColIndex = np.stack((rowsPD, colsPD), axis=-1)

                newPDRowColIndexFlag = array_intersect(oldPDRowColIndex, newWRowColIndex)  # careful about order

                valsPDNew = valsPD[newPDRowColIndexFlag]
                rowsPDNew = rowsPD[newPDRowColIndexFlag]
                colsPDNew = colsPD[newPDRowColIndexFlag]

                self.pdw[i] = coo_matrix((valsPDNew, (rowsPDNew, colsPDNew)),
                                         (self.dimensions[i - 1], self.dimensions[i])).tocsr()

                if(i==1):
                    w1 = coo_matrix((valsWNew, (rowsWNew, colsWNew)),
                                       (self.dimensions[i - 1], self.dimensions[i])).tocsr()
                    self.inputLayerConnections.append(coo_matrix((valsWNew, (rowsWNew, colsWNew)),
                                       (self.dimensions[i - 1], self.dimensions[i])).getnnz(axis=1))
                    np.savez_compressed(self.save_filename + "_input_connections.npz",
                                        inputLayerConnections=self.inputLayerConnections)
                else:
                    w2 = coo_matrix((valsWNew, (rowsWNew, colsWNew)),
                                       (self.dimensions[i - 1], self.dimensions[i])).tocsr()


                # add new random connections
                keepConnections = np.size(rowsWNew)
                lengthRandom = valsW.shape[0] - keepConnections
                randomVals = np.random.randn(lengthRandom) / 10
                zeroVals = 0 * randomVals  # explicit zeros

                # adding  (wdok[ik,jk]!=0): condition
                while (lengthRandom > 0):
                    ik = np.random.randint(0, self.dimensions[i - 1], size=lengthRandom, dtype='int32')
                    jk = np.random.randint(0, self.dimensions[i], size=lengthRandom, dtype='int32')

                    randomWRowColIndex = np.stack((ik, jk), axis=-1)
                    randomWRowColIndex = np.unique(randomWRowColIndex, axis=0)  # removing duplicates in new rows&cols
                    oldWRowColIndex = np.stack((rowsWNew, colsWNew), axis=-1)

                    uniqueFlag = ~array_intersect(randomWRowColIndex, oldWRowColIndex)  # careful about order & tilda

                    ikNew = randomWRowColIndex[uniqueFlag][:, 0]
                    jkNew = randomWRowColIndex[uniqueFlag][:, 1]
                    # be careful - row size and col size needs to be verified
                    rowsWNew = np.append(rowsWNew, ikNew)
                    colsWNew = np.append(colsWNew, jkNew)

                    lengthRandom = valsW.shape[0] - np.size(rowsWNew)  # this will constantly reduce lengthRandom

                # adding all the values along with corresponding row and column indices
                valsWNew = np.append(valsWNew, randomVals)
                # valsPDNew=np.append(valsPDNew, zeroVals)
                if (valsWNew.shape[0] != rowsWNew.shape[0]):
                    print("not good")
                self.w[i] = coo_matrix((valsWNew, (rowsWNew, colsWNew)),
                                       (self.dimensions[i - 1], self.dimensions[i])).tocsr()

                # print("Number of non zeros in W and PD matrix after evolution in layer",i,[(self.w[i].data.shape[0]), (self.pdw[i].data.shape[0])])

                t_ev_2 = datetime.datetime.now()
                print("Weights evolution time for layer",i,"is", t_ev_2 - t_ev_1)
        return w1 , w2

    def predict(self, x):
        _, a = self._feed_forward(x)
        a_hidden = a[self.n_layers-1]
        a_output = a[self.n_layers]
        del a
        return a_hidden, a_output


# **Utils.py**

This code is an adjustment of the existing code that stems from: Atashgahi, Z., Sokar, G., van der Lee, T., Mocanu, E., Mocanu, D. C., Veldhuis, R., & Pechenizkiy, M. (2020). Quick and robust feature selection: the strength of energy-efficient sparse training for autoencoders. arXiv preprint arXiv:2012.00560.

In [ ]:

def load_data(name):

    if name == "coil20":
        mat = scipy.io.loadmat("/content/gdrive/MyDrive/datasets/Coil20.mat")
        print(mat['images'])
        X = mat['images']
        y = mat['labels']
        y = np.array([int(label[3:]) for label in y])
        print("labels", y)

        num_samples, height, width = X.shape
        X = X.reshape((num_samples, height * width))
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
        scaler = preprocessing.StandardScaler().fit(X_train)
        print("type X_train", type(X_train))
        print("type y_train", type(y_train))
        print("first element X_train", X_train[0])
        print("first element y_train", y_train[0])
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

    elif name == "MNIST":
        filename = f"/content/gdrive/MyDrive/datasets/MNIST_{target_gini}.npz"
        data = np.load(filename)

        X_train = data['X_train']
        y_train = data['y_train']
        X_test = data['X_test']
        y_test = data['y_test']

    return X_train, y_train, X_test, y_test


def check_path(filename):
    import os
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise


# **Train sparse DAE**

The given code is an implementation of a Sparse Denoising Autoencoder (DAE). The DAE is a type of neural network that is trained to reconstruct its input data by learning a compressed representation in an intermediate hidden layer.

This code is an adjustment of the existing code that stems from: Atashgahi, Z., Sokar, G., van der Lee, T., Mocanu, E., Mocanu, D. C., Veldhuis, R., & Pechenizkiy, M. (2020). Quick and robust feature selection: the strength of energy-efficient sparse training for autoencoders. arXiv preprint arXiv:2012.00560.

and

Mocanu, D. C., Mocanu, E., Stone, P., Nguyen, P. H., Gibescu, M., & Liotta, A. (2018). Scalable training of artificial neural networks with adaptive sparse connectivity inspired by network science. Nature communications, 9(1), 2383.

In [ ]:
def train_sparse_DAE(dataset, epoch):

  parser = argparse.ArgumentParser(description='Train Sparse DAE')
  #parser.add_argument('--dataset_name', type=str, required=True, help='dataset_name')
  #parser.add_argument("--epoch", help="epoch", type=int)
  #args = parser.parse_args()
  #dataset_name = args.dataset_name


  strtime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
  noise_factor = 0.2
  weightDecay = 0.00001
  noHiddenNeuronsLayer=1000
  batchSize=100
  dropoutRate=0.2
  learningRate=0.01
  momentum=0.9
  epsilon = 17 #a given sparsity level
  zeta = 0.2
  adjusted_X_train, adjusted_y_train, adjusted_X_test, adjusted_y_test = load_data(dataset_name)
  adjusted_y_train = np.array(adjusted_y_train)
  unique_labels = np.unique(adjusted_y_train)
  label_counts = np.array([np.count_nonzero(adjusted_y_train == label) for label in unique_labels])
  relative_distribution = label_counts / len(adjusted_y_train)
  print("relative_distribution", relative_distribution)
  print("The gini-coefficient of the adjusted trainingsdata =")
  print(calculate_gini(relative_distribution))
  print("The amount of cases in the trainingsdata =")
  print(adjusted_y_train.shape)

#in range 5 so the mean and standard deviation can be taken
  for i in range(5):
    filename = "/content/gdrive/MyDrive/results/" + dataset_name+"/"+str(strtime)+"_target_gini_"+str(target_gini)+"_epsilon_" + str(epsilon) + "_zeta_" + str(zeta) + "/run " +str(i)+ "/"
    check_path(filename)
    print("*******************************************************************************")
    print("Dataset = ", dataset_name)
    print("epsilon = ", epsilon)
    print("zeta = ", zeta)
    print("3. Imbalanced Dataset is made")
    noTrainingSamples = adjusted_X_train.shape[0]


    start = time.time()
    set_dae = Sparse_DAE((adjusted_X_train.shape[1], noHiddenNeuronsLayer, adjusted_X_train.shape[1]), (Sigmoid, Linear), epsilon=epsilon)

    set_dae.fit(adjusted_X_train, adjusted_y_train.ravel(), adjusted_X_test, adjusted_y_test.ravel(), loss=MSE,
                  epochs=epoch, batch_size=batchSize, learning_rate=learningRate,
                  momentum=momentum, weight_decay=weightDecay, zeta=zeta, dropoutrate=dropoutRate,
                  testing=False, save_filename=filename, noise_factor = noise_factor)
    end = time.time()
    print("running time QS_",epoch,"= ", end - start)

# **Training (Sparse DAE)**

The sparse DAE is used for unsupervised feature learning. Unsupervised feature learning is a machine learning technique where a model learns to extract useful features or representations from unlabeled data without the need for explicit supervision or labeled examples. Autoencoders are neural network models that learn to reconstruct the input data from a compressed representation called the "latent space." After training the SDAE, the code 'QuickSelection' applies feature selection on the learned features to select a subset of the most relevant features.

In [ ]:
train_sparse_DAE(dataset_name, epoch)

relative_distribution [0.70135747 0.0331825  0.0331825  0.0331825  0.0331825  0.0331825
 0.0331825  0.0331825  0.0331825  0.0331825 ]
The gini-coefficient of the adjusted trainingsdata =
0.6013574660633484
The amount of cases in the trainingsdata =
(6630,)
*******************************************************************************
Dataset =  MNIST
epsilon =  13
zeta =  0.2
3. Imbalanced Dataset is made
Create sparse matrix with  22823  connections and  2.9110969387755103 % density level
Create sparse matrix with  22882  connections and  2.9186224489795918 % density level
----------------------------------------------------
epoch  0
Training time:  0:02:38.574642
Weights evolution time for layer 1 is 0:00:00.083262
Weights evolution time for layer 2 is 0:00:00.079001
----------------------------------------------------
epoch  1
Training time:  0:02:36.434393
Weights evolution time for layer 1 is 0:00:00.077709
Weights evolution time for layer 2 is 0:00:00.073649
--------------------

In [ ]:
num_categories = len(set(Y_train))
gini2, distribution2 = gini_algorithm(target_gini, num_categories)
adjusted_X_train, adjusted_y_train, adjusted_X_test, adjusted_y_test = imbalanced_data(X_train, Y_train, X_test, Y_test, distribution2)

NameError: ignored

In [ ]:
adjusted_y_train = np.array(adjusted_y_train)
unique_labels = np.unique(adjusted_y_train)
label_counts = np.array([np.count_nonzero(adjusted_y_train == label) for label in unique_labels])
relative_distribution = label_counts / len(adjusted_y_train)

print("The gini-coefficient of the adjusted trainingsdata =")
calculate_gini(relative_distribution)

In [ ]:
import requests
import time

while True:
    try:
        requests.get('https://www.google.com')
        print("Kept alive.")
    except:
        print("Failed to keep alive.")
    time.sleep(600)

# **fs_utilis**

These functions provide different feature selection methods based on node strength and node degree in input matrices W1 and W2. The specific method is chosen based on the value of the method parameter passed to the feature_selection function.


This code is an adjustment of the existing code that stems from: Atashgahi, Z., Sokar, G., van der Lee, T., Mocanu, E., Mocanu, D. C., Veldhuis, R., & Pechenizkiy, M. (2020). Quick and robust feature selection: the strength of energy-efficient sparse training for autoencoders. arXiv preprint arXiv:2012.00560.

In [ ]:
def feature_selection(W1, W2, k, method):
    if method == "Node Strength(in)":
        indices =  fs_strength_w(sparse.csr_matrix.transpose(W1), k)
    elif method == "Node Strength(out)":
        indices =  fs_strength_w(W2, k)
    elif method == "Node Degree(in)":
        indices =  fs_degree_w(sparse.csr_matrix.transpose(W1), k)
    elif method == "Node Degree(out)":
        indices =  fs_degree_w(W2, k)
    elif method == "Degree(in_out)":
        indices = fs_degree12(sparse.csr_matrix.transpose(W1), W2, k)
    elif method == "Degree(out_in)":
        indices = fs_degree12(W2, sparse.csr_matrix.transpose(W1), k)
    elif method == "Node Strength(in_out)":
        indices = fs_strength12(sparse.csr_matrix.transpose(W1), W2, k)
    elif method == "Node Strength(out_in)":
        indices = fs_strength12(W2, sparse.csr_matrix.transpose(W1), k)

    return indices

def get_degree(W):
    weights = W.tolil()
    num_ws = np.zeros(W.shape[1])
    for i in range(W.shape[1]):
        num_ws[i] = len(np.sum(weights[:,i].data))
    return num_ws

def fs_strength_w(W, k):
    weights = W.tolil()
    abs_w = np.absolute(weights)
    sum_abs_w = np.sum(abs_w, axis = 0)
    new_sum_w = np.zeros(W.shape[1])
    #print(W.shape)
    new_sum_w = np.zeros(W.shape[1])
    for i in range(W.shape[1]):
        new_sum_w[i] = sum_abs_w[0,i]
    indices = new_sum_w.argsort()[-k:][::-1]
    return indices

def fs_degree_w(W, k):
    num_ws = np.zeros(W.shape[1])
    for i in range(W.shape[1]):
        num_ws[i] = len(W[:,i].data)
    fs_indices = num_ws.argsort()[-k:][::-1]
    return fs_indices

def fs_degree12(W1, W2, k):
    weights = W1.tolil()
    num_ws1 = np.zeros(W1.shape[1])
    for i in range(W1.shape[1]):
        num_ws1[i] = len(np.sum(weights[:,i].data))

    weights = W2.tolil()
    num_ws2 = np.zeros(W2.shape[1])
    for i in range(W2.shape[1]):
        num_ws2[i] = len(np.sum(weights[:,i].data))

    num_ws = num_ws1 + num_ws2
    c = int(num_ws.shape[0]*2/3)
    indices = num_ws.argsort()[-c:][::-1]

    weights = W1.tolil()
    num_ws = np.zeros(W1.shape[1])
    for i in range(W1.shape[1]):
        num_ws[i] = len(np.sum(weights[indices,i].data))
    fs_indices = num_ws.argsort()[-k:][::-1]
    return fs_indices


def fs_strength12(W1, W2, k):

    weights = W1.tolil()
    abs_w = np.absolute(weights)
    sum_abs_w = np.sum(abs_w, axis = 1)
    sum_abs_w = np.transpose(sum_abs_w)
    new_sum_w1 = np.zeros(W1.shape[0])
    for i in range(W1.shape[0]):
        new_sum_w1[i] = sum_abs_w[0,i]

    weights = W2.tolil()
    abs_w = np.absolute(weights)
    sum_abs_w = np.sum(abs_w, axis = 1)
    sum_abs_w = np.transpose(sum_abs_w)
    new_sum_w2 = np.zeros(W2.shape[0])
    for i in range(W2.shape[0]):
        new_sum_w2[i] = sum_abs_w[0,i]

    new_sum_w = new_sum_w1 + new_sum_w1
    c = int(new_sum_w.shape[0]*0.5)
    indices1 = new_sum_w.argsort()[-c:][::-1]

    weights = W1.tolil()
    abs_w = np.absolute(weights)
    sum_abs_w = np.sum(abs_w[indices1,:], axis = 0)
    sum_abs_w = np.transpose(sum_abs_w)
    new_sum_w = np.zeros(W1.shape[1])

    for i in range(W1.shape[1]):
        new_sum_w[i] = sum_abs_w[i,0]

    indices = new_sum_w.argsort()[-k:][::-1]

    return indices


# **QuickSelection**



"cacc" stands for "**clustering accuracy**" and represents the accuracy of the clustering results obtained after applying feature selection. It is the accuracy metric calculated by the evaluation function. Clustering is an unsupervised learning task that groups similar samples together based on their feature similarities. Unlike classification, clustering does not have predefined class labels. Therefore, the evaluation of clustering accuracy is relative to the clustering algorithm and the underlying data distribution. It is not directly comparable to classification accuracy.

"Etacc" stands for "**Extra Trees accuracy**" and represents the accuracy of the classification results obtained using the Extra Trees Classifier. It measures how well the classifier can predict the class labels based on the selected features. A high ETacc indicates that the selected features are effective in distinguishing different classes, resulting in accurate classification.The *eval_subset* function evaluates the performance of a subset of features selected using the SDAE.

This code is an adjustment of Atashgahi, Z., Sokar, G., van der Lee, T., Mocanu, E., Mocanu, D. C., Veldhuis, R., & Pechenizkiy, M. (2020). Quick and robust feature selection: the strength of energy-efficient sparse training for autoencoders. arXiv preprint arXiv:2012.00560.

In [ ]:
def best_map(l1, l2):
    """
    Permute labels of l2 to match l1 as much as possible
    """
    if len(l1) != len(l2):
        print("L1.shape must == L2.shape")
        exit(0)

    label1 = np.unique(l1)
    n_class1 = len(label1)

    label2 = np.unique(l2)
    n_class2 = len(label2)

    n_class = max(n_class1, n_class2)
    G = np.zeros((n_class, n_class))

    for i in range(0, n_class1):
        for j in range(0, n_class2):
            ss = l1 == label1[i]
            tt = l2 == label2[j]
            G[i, j] = np.count_nonzero(ss & tt)

    row_ind, col_ind = linear_sum_assignment(-G)

    new_l2 = np.zeros(l2.shape)
    for i in range(len(col_ind)):
        new_l2[l2 == label2[col_ind[i]]] = label1[row_ind[i]]

    return new_l2.astype(int)


def evaluation(X_selected, n_clusters, y):
    """
    This function calculates ARI, ACC and NMI of clustering results
    Input
    -----
    X_selected: {numpy array}, shape (n_samples, n_selected_features}
            input data on the selected features
    n_clusters: {int}
            number of clusters
    y: {numpy array}, shape (n_samples,)
            true labels
    Output
    ------
    nmi: {float}
        Normalized Mutual Information
    acc: {float}
        Accuracy
    """
    k_means = KMeans(n_clusters=n_clusters)

    k_means.fit(X_selected)
    y_predict = k_means.labels_



    # calculate ACC
    y_permuted_predict = best_map(y, y_predict)
    acc = accuracy_score(y, y_permuted_predict)

    return acc

def eval_subset(train, test):
    n_clusters = len(np.unique(train[2]))

    clf = ExtraTreesClassifier(n_estimators = 50, n_jobs = -1)
    clf.fit(train[0], train[2])
    DTacc = float(clf.score(test[0], test[2]))

    max_iters = 10
    cacc = 0.0
    for iter in range(max_iters):
        acc = evaluation(train[0], n_clusters = n_clusters, y = train[2]) #We repeat the K-means algorithm 10 times and report the average clustering results since K-means may converge to a local optimal
        cacc += acc / max_iters
    return  DTacc,float(cacc)




import time
methods = ['Node Strength(in)']
path = "/content/gdrive/MyDrive/results/"+dataset_name+"/"
#print("Data = ", dataset_name)
#X_train, Y_train, X_test, Y_test = load_data(dataset_name)
#num_categories = len(set(Y_train))
#gini, distribution = gini_algorithm(target_gini, num_categories)
#adjusted_X_train, adjusted_y_train, adjusted_X_test, adjusted_y_test = imbalanced_data(X_train, Y_train, X_test, Y_test, distribution)


print("Performing feature selection on ", dataset_name)
print("Dataset shape:")
print(adjusted_X_train.shape)
print(adjusted_y_train.shape)
print(adjusted_X_test.shape)
print(adjusted_y_test.shape)

k =50

for file in os.listdir(path):

    cnt = 0
    ls_acc = []
    ls_acc2 = []
    print("------------------------------------------------------------")
    print("Quick_Selection results:")
    for run in os.listdir(path+file+"/"):
        print("Run %d:" % cnt, end = "")
        cnt += 1
        w1_10  = scipy.sparse.load_npz(path+file+"/"+run+"/w1.npz") # loads a NumPy-compatible sparse matrix saved in the compressed NPZ format.
        w2_10  = scipy.sparse.load_npz(path+file+"/"+run+"/w2.npz") #
        for j in range(len(methods)):
            indices  = feature_selection(w1_10, w2_10, k, methods[j])
            DTacc, cacc = eval_subset([adjusted_X_train[:,indices], adjusted_X_train, adjusted_y_train.ravel()],
                            [adjusted_X_test[:, indices], adjusted_X_test,  adjusted_y_test.ravel()])
            print("ETacc=%.4f | cacc=%.4f |" % ( DTacc,cacc))
            ls_acc.append(cacc)
            ls_acc2.append(DTacc)
    ls_acc = np.array(ls_acc)
    mean = np.mean(ls_acc)
    std = np.std(ls_acc)
    print("cacc | mean=%.4f |  std=%.4f " % (mean, std))
    ls_acc2 = np.array(ls_acc2)
    mean = np.mean(ls_acc2)
    std = np.std(ls_acc2)
    print("Etacc | mean=%.4f |  std=%.4f"   % (mean, std))
    print("shape indices", indices.shape)


In [ ]:
print(adjusted_X_train.shape, adjusted_y_train.shape)
print(adjusted_X_test.shape, adjusted_y_test.shape)

x = selector.get_support(indices= False)

print(x) #784 items

#hoe te zorgen dat x deel twee wordt van adjusted_X_train zodat die gebruikt kunnen worden voor de analyse hieronder

In [ ]:
features_AEC = selector.transform(adjusted_X_train)

print(features_AEC[1])
print(features_AEC.shape) #200 is het aantal features dat geselecteerd moest worden, 784 is het aantal features origineel

In [ ]:
def evaluation(X_selected, n_clusters, y):
    """
    This function calculates ARI, ACC and NMI of clustering results
    Input
    -----
    X_selected: {numpy array}, shape (n_samples, n_selected_features}
            input data on the selected features
    n_clusters: {int}
            number of clusters
    y: {numpy array}, shape (n_samples,)
            true labels
    Output
    ------
    nmi: {float}
        Normalized Mutual Information
    acc: {float}
        Accuracy
    """
    k_means = KMeans(n_clusters=n_clusters)

    k_means.fit(X_selected)
    y_predict = k_means.labels_


    # calculate ACC
    y_permuted_predict = best_map(y, y_predict)
    acc = accuracy_score(y, y_permuted_predict)

    return acc




def eval_subset(train, test):
    n_clusters = len(np.unique(train[2]))

    clf = ExtraTreesClassifier(n_estimators = 50, n_jobs = -1)
    clf.fit(train[0], train[2])
    DTacc = float(clf.score(test[0], test[2]))

    max_iters = 10
    cacc = 0.0
    for iter in range(max_iters):
        acc = evaluation(train[0], n_clusters = n_clusters, y = train[2])
        cacc += acc / max_iters
    return  DTacc,float(cacc)

DTacc, cacc = eval_subset([adjusted_X_train[:, selector.get_support(indices = True)], adjusted_X_train, adjusted_y_train.ravel()], [adjusted_X_test[:, selector.get_support(indices = True)], adjusted_X_test,  adjusted_y_test.ravel()])
print("ETacc=%.4f | cacc=%.4f |" % ( DTacc,cacc))

In [ ]:
import requests
import time

while True:
    try:
        requests.get('https://www.google.com')
        print("Kept alive.")
    except:
        print("Failed to keep alive.")
    time.sleep(600)